In [46]:
import glob
import pandas as pd

In [52]:
path_folder = "../../datasets/clipart/*/*.jpg"

dictionary = {
    "image_path": [],
    "type dataset": [],
    "classes": []
}

for image_path in glob.glob(path_folder):
    image_path = image_path.replace("../../", "")
    dictionary["image_path"].append(image_path)
    dictionary['type dataset'].append(image_path.split("/")[1])
    dictionary['classes'].append(image_path.split("/")[2])
    

In [53]:
df = pd.DataFrame(dictionary)
df

,image_path,type dataset,classes
0,datasets/clipart/lollipop/clipart_178_000023.jpg,clipart,lollipop
1,datasets/clipart/lollipop/clipart_178_000037.jpg,clipart,lollipop
2,datasets/clipart/lollipop/clipart_178_000036.jpg,clipart,lollipop
3,datasets/clipart/lollipop/clipart_178_000022.jpg,clipart,lollipop
4,datasets/clipart/lollipop/clipart_178_000008.jpg,clipart,lollipop
...,...,...,...
48828,datasets/clipart/dumbbell/clipart_101_000302.jpg,clipart,dumbbell
48829,datasets/clipart/dumbbell/clipart_101_000316.jpg,clipart,dumbbell
48830,datasets/clipart/dumbbell/clipart_101_000128.jpg,clipart,dumbbell
48831,datasets/clipart/dumbbell/clipart_101_000100.jpg,clipart,dumbbell


In [54]:
def distribution_tenant(df):
    dictionary = {
        "Classes": [],
        "Number Images": []
    }
    
    for class_name in set(df.classes):
        dictionary['Classes'].append(class_name)
        dictionary['Number Images'].append(df[df.classes == class_name].shape[0])
    
    return pd.DataFrame(dictionary)

In [55]:
distribution_tenant(df).sort_values(by=['Number Images'], ascending=False).to_csv("./distribution_classes.csv", index=False)

In [61]:
# read information
dictionary_train = {
    "image_path": [],
    "class_name": []
}
f = open("../../datasets/clipart_info/clipart_train.txt", "r")
for x in f:
    x = x.replace("\n","")
    image_path = x.split(" ")[0]
    class_name = image_path.split("/")[1]
    dictionary_train['image_path'].append(image_path)
    dictionary_train['class_name'].append(class_name)

In [64]:
df_train = pd.DataFrame(dictionary_train)
df_train.to_csv("../../datasets/clipart_info/train.csv", index=False)

In [67]:
# read information
dictionary_test = {
    "image_path": [],
    "class_name": []
}
f = open("../../datasets/clipart_info/clipart_test.txt", "r")
for x in f:
    x = x.replace("\n","")
    image_path = x.split(" ")[0]
    class_name = image_path.split("/")[1]
    dictionary_test['image_path'].append(image_path)
    dictionary_test['class_name'].append(class_name)

In [68]:
df_test = pd.DataFrame(dictionary_test)
df_test.to_csv("../../datasets/clipart_info/test.csv", index=False)

In [91]:
import sys
sys.path.insert(0, "../base_line/")

from efficienet import Net
from torchsummary import summary

In [92]:
model = Net()

for param in model.parameters():
    param.requires_grad = False
for param in model.backbone._conv_head.parameters():
    param.requires_grad = True
for param in model.backbone._fc.parameters():
    param.requires_grad = True

Loaded pretrained weights for efficientnet-b0


In [94]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Check image

In [21]:
import pydicom as dicom
import cv2

In [23]:
# image_path = "/Users/tuananh/Downloads/datasets/1.2.392.200046.100.14.10686001178863181887404916406881043409_left.dcm"
# ds = dicom.dcmread(image_path)
# pixel_array_numpy = ds.pixel_array

In [45]:
# from PIL import Image
# im = Image.open("/Users/tuananh/Downloads/datasets/1.2.392.200046.100.14.709086559867564263504423229097465362078_left.gif")

# try:
#     while 1:
#         im.seek(im.tell()+1)
        
# except EOFError:
#     pass # end of sequence

In [44]:
# from PIL import Image, ImageSequence

# im = Image.open("/Users/tuananh/Downloads/datasets/1.2.392.200046.100.14.709086559867564263504423229097465362078_left.gif")

# index = 1
# for frame in ImageSequence.Iterator(im):
#     frame.save("frame%d.png" % index)
#     index += 1
#     break

In [40]:
import pydicom as dicom
import glob
import numpy as np
from tqdm import tqdm

path_as = "/Users/tuananh/Downloads/miss_189/"
path_ai_scaler = "/Users/tuananh/Downloads/out2/*.dcm"

ai_scaler_dcm = glob.glob(path_ai_scaler)

In [41]:
dictionary = {
    "image_id": [],
    "distance": []
}

In [42]:
for image_path_ai_scaler in tqdm(ai_scaler_dcm, total=len(ai_scaler_dcm)):
    image_id = image_path_ai_scaler.split("/")[-1].split("_right_")[0]
    image_path_as = path_as + f"{image_id}.dcm"
    try:
        ds_ai_scaler = dicom.dcmread(image_path_ai_scaler)
        image_ai_scaler = ds_ai_scaler.pixel_array

        ds_as = dicom.dcmread(image_path_as)
        image_as = ds_as.pixel_array

        if image_ai_scaler.shape[0] != image_as.shape[0] or image_ai_scaler.shape[1] != image_as.shape[1]:
            dictionary["image_id"].append(image_id)
            dictionary["distance"].append(-1)
        else:
            dictionary["image_id"].append(image_id)
            dictionary["distance"].append(np.sum(image_ai_scaler - image_as))
    except:
        dictionary["image_id"].append(image_id)
        dictionary["distance"].append(np.nan)

 76%|███████▌  | 144/189 [05:56<02:38,  3.53s/it]/Users/tuananh/miniconda3/envs/dicom_file/lib/python3.6/site-packages/pydicom/filereader.py:440: UserWarning: End of file reached before delimiter (fffe, e0dd) found in file /Users/tuananh/Downloads/out2/1.2.392.200046.100.14.74734413188286174604827912507936614830_right_1.2.392.200046.100.14.567077676569214124214893545472926360616.dcm
  warnings.warn(msg, UserWarning)
100%|██████████| 189/189 [07:59<00:00,  2.54s/it]


In [48]:
import pandas as pd
df = pd.DataFrame(dictionary)
df

,image_id,distance
0,1.2.392.200046.100.14.435953841381033664182198...,-1.0
1,1.2.392.200046.100.14.125731478164606132829829...,0.0
2,1.2.392.200046.100.14.677163186447127745222001...,0.0
3,1.2.392.200046.100.14.365905801015524484269252...,0.0
4,1.2.392.200046.100.14.611691663363053369124180...,0.0
...,...,...
184,1.2.392.200046.100.14.409622991622341805812522...,0.0
185,1.2.392.200046.100.14.470067761410355946798114...,0.0
186,1.2.392.200046.100.14.237779557168211912558127...,0.0
187,1.2.392.200046.100.14.160050538223674831155764...,0.0


In [90]:
df_hanh = pd.read_excel('/Users/tuananh/Downloads/compare_name (1).xlsx', engine='openpyxl')
df_ta = pd.read_csv("/Users/tuananh/Downloads/result_matching.csv")

In [91]:
def complex_function(x):
    return x["AS_Instance ID"].replace("_left.gif", "")
        
df_hanh["image_id"] = df_hanh.apply(lambda x: complex_function(x), axis=1)

In [103]:
df_ta_filter = df_ta[df_ta.distance != 0]
df_hanh_filter = df_hanh[df_hanh.match != "Giống"]

set(df_hanh_filter[~df_hanh_filter.image_id.isin(list(df_ta_filter.image_id))]["AS_Instance ID"])

set()

In [104]:
df_ta_filter.shape

(49, 2)

In [105]:
df_hanh_filter.shape

(46, 9)

# convert image

In [16]:
import pydicom
import glob
import numpy as np
from tqdm import tqdm
import cv2

path_folder = "/Users/tuananh/Downloads/Batch 2/images/*.dcm"
path_save = "/Users/tuananh/Downloads/Batch 2/image_batch_2/"

for path_file in tqdm(glob.glob(path_folder)):
    ds = pydicom.dcmread(path_file)

    shape = ds.pixel_array.shape

    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    
    image_name = path_file.split("/")[-1].replace(".dcm", ".jpeg")
    cv2.imwrite(path_save + image_name, image_2d_scaled)


100%|██████████| 49/49 [00:09<00:00,  5.04it/s]
